In [280]:
using Random, LinearAlgebra, TensorToolbox, Combinatorics, TensorOperations

In [575]:
function makeRankedTensor(L::Vector, A::Array, d::Int)
    A = kronMat(A, d)
    return reshape(sum(kronMat(A, d) .* L', dims=2), tuple(repeat([n], d)...))
end;

function randomTensor(n::Int, d::Int; real::Bool=false)
    T_ = randn(tuple(repeat([n], d)...)...)
    if !real
        Q = im*randn(tuple(repeat([n], d)...)...)
        T_ = T_+Q
    end;
    T = copy(T_)
    perms = permutations(1:d)
    for perm in perms
        if perm != 1:d 
            T = T + permutedims(T_, perm)
        end;
    end;
    return T/factorial(d)
end;

function complexGaussian(m, n)
    A = randn(m, n)
    B = randn(m, n)
    return A + im*B
end;

function randomRankedTensor(n, d, r; real=false)
    if !real
        A = complexGaussian(n, r)
    else 
        A = randn(n, r)
    end;
    L = ones(r)
    A_ = copy(A)
    A1 = A_[1, :]
    L_ = zeros(eltype(A_), r)
    for i=1:r
        A_[:, i] ./= A1[i]
        L_[i] = A1[i]^d
    end;
    return makeRankedTensor(L, A, d), A_, L_
end;

function contract(T::Array, V::Array)
    d1 = length(size(T))
    d2 = length(size(V))
    return ncon((T, V), (vcat(collect(1:d2), -collect(1:d1-d2)), collect(1:d2)))
end;

function jennrich(T::Array; tol=1e-10)
    Tsize = size(T)
    n = Tsize[1]
    d = length(Tsize)

    # draw random vectors or matrices for contraction
    B = randn(repeat([n], Int(iseven(d))+1)...);
    C = randn(repeat([n], Int(iseven(d))+1)...);

    # contract to tensor with largest even order strictly less than the order of d
    # d odd -> order n^{d-1} tensor
    # d even -> order n^{d-2} tensor 
    T1 = contract(T, B);
    T2 = contract(T, C);

    delt = Int(floor((d-1)/2))

    # flatten to matrices
    T1flat = reshape(T1, (n^delt, n^delt));
    T2flat = reshape(T2, (n^delt, n^delt));

    # obtain the factors
    # T real -> true factors are obtained, possibly up to sign
    # T complex -> true factors are obtained, but up to a complex scalar factor of norm 1 
    w, Ahat_ = eigen(T1flat*pinv(T2flat))
    Ahat_ = Ahat_[:, abs.(w).>tol]
    r = last(size(Ahat_))
    Ahat = zeros(eltype(Ahat_), (n, r))
    for i=1:r
        a = Ahat_[:, i]
        U, s, _ = svd(reshape(a, (n, n^(delt-1))))
        Ahat[:, i] = U[:, abs.(s).>tol]
    end;
    Ahat = dehomogenize!(Ahat)

    # obtain L and deflate
    low = Int(floor(d/2))
    high = Int(ceil(d/2))
    Tflat = reshape(T, (n^low, n^high))
    Alow = kronMat(Ahat, low)
    Ahigh = kronMat(Ahat, high)

    Lhat = diag(pinv(Alow)*Tflat*pinv(transpose(Ahigh)))

    return Ahat, Lhat
end;

function dehomogenize!(A)
    for col in eachcol(A)
        col ./= col[1]
    end;
    return A
end;
dehomogenize(A) = dehomogenize!(copy(A));

function normcol!(A)
    for col in eachcol(A)
        col ./= norm(col)
    end;
    return A
end;
normcol(A) = normcol!(copy(A));

function kronMat(A::Matrix, d)
    n, r = size(A)
    B = zeros(eltype(A), (n^d, r))
    for i=1:r 
        B[:, i] = kron(ntuple(x->A[:, i], d)...)
    end;
    return B
end;


In [576]:
n = 5;
d = 5;

In [589]:
r = 6;
T, A, L = randomRankedTensor(n, d, r; real=false);
Ahat, Lhat = jennrich(T);

In [590]:
A

5×6 Matrix{ComplexF64}:
       1.0+0.0im             1.0-0.0im        …        1.0-1.7515e-17im
 -0.943229+0.369542im  -0.290456+0.0890217im      0.645893-0.980246im
  0.427695-1.45498im   -0.841642+0.405686im         1.7418-3.05544im
 -0.817363-2.18054im     0.32911+0.471043im        6.11404+6.36488im
   -1.1666+0.279473im   0.772352-0.0145782im     -0.122255-1.2682im

In [591]:
L

6-element Vector{ComplexF64}:
 -0.024372019174050314 + 0.02475612333018387im
     16.73846869136191 + 22.59978762226049im
    1.0997305873080043 - 0.15727315149992127im
    0.9729356448000009 + 2.1622251220945516im
    -7.776181019047463 - 73.02198231806913im
 -0.006326577405817072 - 0.006697938533394753im

In [592]:
Ahat

5×6 Matrix{ComplexF64}:
       1.0+0.0im             1.0+3.63031e-32im  …        1.0-0.0im
  0.645893-0.980246im   0.145763-0.438556im        -0.290456+0.0890217im
    1.7418-3.05544im    0.592308+0.697007im        -0.841642+0.405686im
   6.11404+6.36488im   0.0655564-0.153708im          0.32911+0.471043im
 -0.122255-1.2682im    -0.281083-0.107152im         0.772352-0.0145782im

In [593]:
Lhat

6-element Vector{ComplexF64}:
 -0.006326577405817019 - 0.006697938533394913im
    -7.776181019047455 - 73.0219823180693im
    0.9729356448000003 + 2.1622251220945494im
    1.0997305873079848 - 0.15727315149988874im
 -0.024372019174050415 + 0.024756123330183413im
    16.738468691361863 + 22.599787622260468im